# Simulation

In [1]:
# -*- coding: utf-8 -*-
"""
This script will create simulations of our variables
"""

import numpy as np

def multiBrownian(n, dim, T):
    '''
    A multidimensional independent Brownian motion 
    '''
    dt = T/n
    Z = np.sqrt(dt)*np.random.randn(dim, n)
    return np.cumsum(Z, axis = 1)
    
def reshape(array, dim):
    if type(array) == float or type(array) == int:
        array = array*np.ones(dim).reshape((dim,1))
    else:
        array = np.array(array).reshape((dim,1))
        
    return array
    
def geometricBM (n, dim, T, S0, r, delta, sigma, pho):
    '''
    This function will simulate a geometric BM
    
    S0 -> initial Value
    r -> risk free interest rate
    delta -> dividends yields
    sigma -> volatility
    pho -> Brownian Motion correlations
    '''
    delta = reshape(delta, dim)
    sigma = reshape(sigma, dim)
    S0 = reshape(S0, dim)
        
    Tp = np.arange(0, T, T/n)*np.ones((dim,n))
    
    mt = r - delta - sigma**2/2
    
    ms = sigma*sigma.transpose()
    Pho = pho*np.ones((dim, dim)) - (pho-1)*np.diag(np.ones(dim))
    ms = ms*Pho
    
    ms = np.linalg.cholesky(ms)
    
    Z = multiBrownian(n, dim, T)
    
    return S0 * np.exp(mt * Tp + np.matmul(ms, Z))
    
def simulate_x (M, n, dim, T, S0, r, delta, sigma, pho):
    '''
    This function will generate M copies of the geometric BM of dimension dim 
    and time step T/n
    
    S0 -> initial Value
    r -> risk free interest rate
    delta -> dividends yields
    sigma -> volatility
    pho -> Brownian Motion correlations
    '''
    return np.array([geometricBM (n, dim, T, S0, r, delta, sigma, pho) \
                     for i in range(M)])
    



# Training

In [2]:
# -*- coding: utf-8 -*-
"""
Created on Tue Apr 14 02:58:26 2020

@author: raphaelfeijao
"""
import simulation as sl
import tensorflow as tf
import numpy as np

'''
Constants
For (M = 1,000,000, d = 2, n = 1024), it takes 285 seconds == 4.75 minutes
'''
M = 1000
n = 9
d = 2
T = 3
S0 = 100
r = 0.05
delta = 0.1
sigma = 0.2
pho = 0

X = sl.simulate_x (M, n+1, d, T, S0, r, delta, sigma, pho)

'''
Payoff
'''
K = 10
def payoff (tau, x, K = K, T = T, n=n, r=r):
    '''
    The payoff of the option
    Tau will be one of the positions in the array of x
    '''
    P = tf.math.reduce_max(x, axis = 0) - K
    I = tf.math.greater(P[tau],0)
    
    P = tf.cond(I, lambda: P[tau], lambda: tf.convert_to_tensor(0.0, dtype = tf.float64))
    
    return tf.convert_to_tensor(np.exp(-r * tau*T/n), dtype = tf.float64)*P


'''
Training
For each step from 0 to N-1 we have to train a NN. We will have N neural nets
'''

class model:
    
    def __init__(self):
        #xavier=tf.keras.initializers.GlorotUniform()
        self.l1=tf.keras.layers.Dense(2,activation=tf.nn.relu,input_shape=[1000,2,10], dtype = tf.float64)
        self.l2=tf.keras.layers.Dense(1,activation=tf.nn.relu, dtype = tf.float64)
        self.out=tf.keras.layers.Dense(1,activation = tf.nn.sigmoid, dtype = tf.float64)
        self.train_op = tf.keras.optimizers.Adagrad(0.1)
        self.dim = 1000
    # Running the model
    def run(self,X): 
        boom=self.l1(X)
        boom1=self.l2(boom)
        boom2=self.out(boom1)
        return boom2
      
    #Custom loss fucntion
    #Change this for each n
    def get_loss(self,X):
        boom=self.l1(X)
        boom1=self.l2(boom)
        boom2=self.out(boom1)
        r = 0
        for i in range(self.dim):
            r += payoff(n-1,X[i])* boom2 + payoff(n,X[i])*(1-boom2)
        return -r/self.dim
      
    # get gradients
    def get_grad(self,X):
        with tf.GradientTape() as tape:
            tape.watch(self.l1.variables)
            tape.watch(self.l2.variables)
            tape.watch(self.out.variables)
            L = self.get_loss(X)
            g = tape.gradient(L, [self.l1.variables[0],self.l1.variables[1],self.l2.variables[0],self.l2.variables[1],self.out.variables[0],self.out.variables[1]])
        return g
      
    # perform gradient descent
    def network_learn(self,X):
        g = self.get_grad(X)
        print(g)
        print(self.l2.variables[0])
        print(self.l2.variables[1])
        self.train_op.apply_gradients(zip(g, [self.l1.variables[0],self.l1.variables[1],self.l2.variables[0],self.l2.variables[1],self.out.variables[0],self.out.variables[1]]))
        
        
model_test = model()

input_nn = tf.Variable(X, name = 'x')
model_test.network_learn(input_nn)

[<tf.Tensor: id=121190, shape=(10, 2), dtype=float64, numpy=
array([[-6.47963286e-05,  7.18178318e-05],
       [-6.09338874e-05,  6.75368462e-05],
       [-6.08126368e-05,  6.74024565e-05],
       [-6.51873997e-05,  7.22512804e-05],
       [-7.44742139e-05,  8.25444386e-05],
       [-6.94397600e-05,  7.69644379e-05],
       [-8.45644741e-05,  9.37281064e-05],
       [-9.51446198e-05,  1.05454745e-04],
       [-1.03622704e-04,  1.14851537e-04],
       [-1.15454260e-04,  1.27965192e-04]])>, <tf.Tensor: id=121178, shape=(2,), dtype=float64, numpy=array([-6.57041792e-07,  7.28240595e-07])>, <tf.Tensor: id=121176, shape=(2, 1), dtype=float64, numpy=
array([[-6.18168198e-05],
       [-4.62535295e-05]])>, <tf.Tensor: id=121164, shape=(1,), dtype=float64, numpy=array([-5.77128104e-07])>, <tf.Tensor: id=121162, shape=(1, 1), dtype=float64, numpy=array([[-1.60780361e-05]])>, <tf.Tensor: id=121090, shape=(1,), dtype=float64, numpy=array([-0.45092262])>]
<tf.Variable 'dense_1/kernel:0' shape=(2, 1